-----------------------
# **Source**

Content from [Learning Python Design Patterns (Chetan Giridhar, 2016)](https://www.amazon.com.br/dp/B018XYKNOM/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1).

-----------------------

--------
# **Command pattern**

- Is another kind of  *behavioral pattern*.
- In this pattern, an object an object encapsulates all information needed to perform an action or trigger an event at a later time. This information includes the following:
    - The method name.
    - An object that owns the method.
    - Values for method parameters.
    
- Consider a use case for an installation wizard. A typical wizard may contain multiple phases or screens that capture user's preferences. While the user browses through the wizard, s/he makes certain choices. This is the command pattern.
    - A wizard is first launched with an object called **Command**.
    - The preferences or choices made by the user in multiple phases of the wizard are then stored in the **Command** object.
    - When the user click on **Finish** button, the **Command** object runs an *execute()* method which considers all the stored choices and runs the appropriate intallation procedure.
    
- In general, the command pattern work as follows:
    - A **Command** object knows about the **Receiver** objects and invokes a method of the **Receiver** object.
    - Values for parameters of the receiver method are stored in the **Command** object.
    - The invoker knows how to execute a command.
    - The client creates a **Command** object and sets its receiver.
    
 - The main intentions of the command pattern are:
    - Encapsulating a request as an object.
    - Allowing the parameterization of clients with different requests.
    - Allowing to save the requests in a queue.
    - Providing an object-oriented callback
    
- The main applications of this kind of pattern are common in the following scenarios:
    - Parameterizing objects depending on the action to be performed
    - Adding actions to a queue and executing requests at different points.
    - Creating a structure for high-level operations that are based on smaller operations.;

--------------------------------
# **Implementation**

In [1]:
class Wizard():
    def __init__(self, src, rootdir):
        self.choices = []
        self.rootdir = rootdir
        self.src = src
        
    def preferences(self, command):
        self.choices.append(command)
        
    def execute(self):
        for choice in self.choices:
            if list(choice.values())[0]:
                print(f'Copying binaries from {self.src} to {self.rootdir}.')
            else:
                print('No operation.')
                
if __name__ == '__main__':
        ## Client code:
    wizard = Wizard('python.gzip','/usr/bin/')
        
    ## Users chooses to install python only:
    wizard.preferences({'python':True})
    wizard.preferences({'java': False})
        
    wizard.execute()
        

Copying binaries from python.gzip to /usr/bin/.
No operation.


------------------
## **Command pattern UML**

- The command pattern has these five main participants:
    - **Command:** declares an interface to execute an operation
    - **ConcreteCommand:** defines a binding between the **Receiver** object and action.
    - **Receiver:** creates a **ConcreteCommand** object and sets its receiver
    - **Invoker:** asks **ConcreteCommand** to carry out the request.
    - **Client:** knows how to perform the operations associated with carrying out the request.
    
![Command pattern UML](images/command_pattern.jpg)

- The flow is pretty straightforward:
    - The client asks for a command to be executed.
    - The invoker takes the command, ancapsulates it and places it in a queue.
    - The **ConcreteCommand** class is in charge of the requested command and asks the receiver to perform the given action.

- The following code example helps to understand the pattern with all participants involved:

In [2]:
from abc import ABCMeta, abstractmethod


class Command(metaclass=ABCMeta):
    def __init__(self, recv):
        self.recv = recv
        
    def execute(self):
        pass
    
class ConcreteCommand(Command):
    def __init__(self, recv):
        self.recv = recv
        
    def execute(self):
        self.recv.action()
    
class Receiver:
    def action(self):
        print('Receiver action.')
        
class Invoker:
    def command(self, cmd):
        self.cmd = cmd
        
    def execute(self):
        self.cmd.execute()
        
if __name__ == '__main__':
    recv = Receiver()
    cmd = ConcreteCommand(recv)
    invoker = Invoker()
    invoker.command(cmd)
    invoker.execute()

Receiver action.


--------
# **Command pattern in real world**

- In this example we will take up a stock exchange to demonstrate the implementation fot this pattern.
- A stock exchange works more or less like this:
    - A user create orders to buy or sell stocks. But the client don't buy or sell those stocks, you'll need an agent or broker who plays the intermediary between you and the stock exchange.
    - The agent is responsible for taking the client's request to the stock exchange and getting the work done. Imagine that you want to sell a stock on monday morning when the exchange open up.
    - You can still make the request to sell stock on sunday night to your agent even though the exchange is not open yet.
    - The agent then queues your request to be executed on monday morning when the exchange is open for the trading.
    
- This is a classical case for the command pattern.
    
### **Design considerations:**
- Four main participants acts here: **Command**, **ConcreteCommand**, **Invoker** and **Receiver**.
    - In the stock exchange scenario, we should create an **Order** interface that defines the order that a client places. This is the **Command** object. It will be an interface to implement **ConcreteCommand**.
    - The **ConcreteCommand** classes buy or sell a stock. The *execute()* abstract method will be defined here to execute the **Order** class.
    - A class also needs to be defined for the stock exchange. 
    - We should define the **Receiver** class that will actually execute the trade and the agent (know as the invoker) that invokes the order and gets it executed by the receiver.

- The following code represents the abstract class **Order** and the abstract method *execute()*:

In [3]:
from abc import ABCMeta, abstractmethod

class Order(metaclass=ABCMeta):
    
    @abstractmethod
    def execute(self):
        pass

### **ConcreteCommand  design:**

- We will have two main concrete classes: **BuyStockOrder** and **SellStrockOrder** that implement the **Order** interface.
- Both classes use the object of the stock trading system so that they can define appropriate actions for the trading system.
- The *execute()* method of each of these **ConcreteCommand** classes uses the stock trade object to execute the actions to buy and sell


In [4]:
class BuyStockOrder(Order):
    def __init__(self, stock):
        self.stock = stock
        
    def execute(self):
        self.stock.buy()
        
class SellStockOrder(Order):
    def __init__(self, stock):
        self.stock = stock
        
    def execute(self):
        self.stock.sell()
    

### **Receiver design:** 

- The **StockTrade** represents the **Receiver** object in this example. and defines multiple methods (actions) to execute the orders placed by **ConcreteCommand** objects.

- The *buy()* and *sell()* methods are defined by the receiver which are called by **BuyStockOrder** and **SellStockOrder** respectively to buy or sell the stock in the exchange;

In [5]:
class StockTrade:
    def buy(self):
        print('You will buy stocks.')
        
    def sell(self):
        print('You will sell stocks.')
        


### **Invoker design**
- The **Agent** class is the invoker.
- **Agent** ins the intermediary between the client and the **StockExchange** and executes the orders placed by the client.
- Agent defines a data member, *\_\_orderQueue* (a list) that acts as a queue. Any new orders placed by the client are added to the queue.
- The *placeOrder()* method of **Agent** is responsible for queuing the orders and also to execute them.

In [6]:
class Agent:
    def __init__(self):
        self.__orderQueue = []
        
    def placeOrder(self, order):
        self.__orderQueue.append(order)
        order.execute()

### Client design:
- The client first sets its receiver, the **StockTrade** class.
- It creates orders to buy and sell stocks with **BuyStockOrder** and **SellStockOrder** (**ConcreteCommand**) that executes the action on **StockTrade**.
- The invoker objet is created by instantiating the **Agent** class.
- The *placeOrder()* method of **Agent** is used to get the orders that the client places.


In [7]:
if __name__ == '__main__':
    # Client
    stock = StockTrade()
    buyStock = BuyStockOrder(stock)
    sellStock = SellStockOrder(stock)
    
    # Invoker
    agent = Agent()
    agent.placeOrder(buyStock)
    agent.placeOrder(sellStock)

You will buy stocks.
You will sell stocks.


---
# **How is it used?**

- Redo or rollback operations:
    - While implementing the rollback or redo operations, developers can do two different things: take a snapshot of the system and revert to this snapshot.
    - With command pattern you can store te sequence of commands and when asked for a redo you rerun the same actions.
    
- Asynchronous task execution:
    - In distributed systems we often need the facility to perform the asynchronous execution of tasks so that the core service is never blocked in case of more requests
    - In this kind of pattern, the invoker object can maintain a queue of requests and send these thasks to the receiver, so they can be acted on independent of the main application thread.

---
# **Advantages and disadvantages**

- **<font color = blue> Advantages: </font>**
    - It decouples the classes that invoke the operation from the object that knows how to execute the operation.
    - It allows you to create a sequence of commands by providing a queue system.
    - Extensions to add a new command is easy and can be done without changing the existing code.
    - You can also define a rollback system with the command pattern.

- **<font color = red> Advantages: </font>**
    - There are a high number of classes and objects working togeter to achieve a goal. Application developers need to be careful developing these classes correctly.
    
    - Every individual command is a **ConcreteCmmand** class that increases the volume of classes for imlementation and maintenance.

---
# **FAQ**

**Q1** Can there be no **Receiver** and **ConcreteCommand** to implement an execute method?
- Yes. Many software applications use the ommand pattern in this way. The only thing to note here is the interaction between the invoker and receiver. If the receiver is not defined, the level of decoupling goes down. Moreover, the facility to parameterize commands is lost.

**Q2** What data structure do I use to implement the queue mechanism in the invoker object?
- In the stock exchange example that we studied we used a list, however the pattern talks about a stack implementation that is really helpful in case of redo/rollback development.